# Terraform

With Terraform, you can describe your infrastructure using a special declarative language.

The Terraform can manipulate different pieces of software with an API. Each piece of software managed by the Terraform is called a **provider**. A provider could be:

- IaaS: AWS/Azure/GCP.
- PaaS: K8s.
- Specific software: Docker. 

## CLI

To manipulate the terraform use special CLI. The basic commands are:

- `terraform init`: to init the terraform directory.
- `terraform validate`: to check if there are syntax mistakes in your terrform configuration.
- `terraform apply`: to apply the configuration.
- `terraform destroy`: to unroll the infrastructure.

---

The following cell defines a simple Terraform configuration that deploys an `nginx` image in the Docker runtime.

In [7]:
rm -rf /tmp/terraform
mkdir /tmp/terraform
cat << EOF > /tmp/terraform/main.tf
terraform {
  required_providers {
    docker = {
      source = "kreuzwerker/docker"
      version = "~> 3.0.1"
    }
  }
}

provider "docker" {}

resource "docker_image" "nginx" {
  name         = "nginx:latest"
  keep_locally = false
}

resource "docker_container" "nginx" {
  image = docker_image.nginx.image_id
  name  = "terraform_example"
  ports {
    internal = 80
    external = 8000
  }
}
EOF
cd /tmp/terraform

The following cell show the terrafrom initialisation outputs:

In [8]:
terraform init

Initializing the backend...
Initializing provider plugins...
- Finding kreuzwerker/docker versions matching "~> 3.0.1"...
- Installing kreuzwerker/docker v3.0.2...
- Installed kreuzwerker/docker v3.0.2 (self-signed, key ID BD080C4571C6104C)
Partner and community providers are signed by their developers.
If you'd like to know more about provider signing, you can read about it here:
https://developer.hashicorp.com/terraform/cli/plugins/signing
Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration fo

Special folders are created in the terraform directory:

In [9]:
ls -la

total 56
drwxrwxr-x  3 user user  4096 paź 29 07:44 .
drwxrwxrwt 22 root root 36864 paź 29 07:44 ..
-rw-rw-r--  1 user user   400 paź 29 07:44 main.tf
drwxr-xr-x  3 user user  4096 paź 29 07:44 .terraform
-rw-r--r--  1 user user  1337 paź 29 07:44 .terraform.lock.hcl


The validation:

In [10]:
terraform validate

Success! The configuration is valid.



The following code applies the configuration. The `-auto-approve` flag removes the terraform prompts to confirm that you want to apply the changes.

In [11]:
terraform apply -auto-approve &> /dev/null

The specified `nginx` container appeared in the docker runtime.

In [12]:
docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED         STATUS         PORTS                  NAMES
4b2945deff80   9d0e6f6199dc   "/docker-entrypoint.…"   2 seconds ago   Up 2 seconds   0.0.0.0:8000->80/tcp   terraform_example


The following cell shows that after `terraform destroy` the container deployed by terraform disappered:

In [13]:
terraform destroy -auto-approve &> /dev/null
docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


## Output

The output block can be useful for experimentation purposes, as it allows you to easily print values to stdout.

---

The definition of the `"hello_world` output is shown in the following cell.

In [19]:
rm -rf /tmp/terraform
mkdir /tmp/terraform
cat << EOF > /tmp/terraform/main.tf
output "hello_world" {
  value = "Hello, stdout!"
}
EOF
cd /tmp/terraform
terraform init &> /dev/null

In such a case, the `terraform apply` will have an `Outputs` section containing the relevant infromation.

In [16]:
terraform apply -auto-approve


Changes to Outputs:
  + hello_world = "Hello, stdout!"

You can apply this plan to save these new output values to the Terraform state,
without changing any real infrastructure.

Apply complete! Resources: 0 added, 0 changed, 0 destroyed.

Outputs:

hello_world = "Hello, stdout!"


You can also use the `terraform output` command to simply push the outputs to stdout.

In [18]:
terraform output

hello_world = "Hello, stdout!"


## Variables

The `variable` allows you to define the variables that can be set using an external configuration. This allows you to parameterise you configuration.

---

The following cell adds the `variable` parameter and prints its value alongside the output.

In [23]:
rm -rf /tmp/terraform
mkdir /tmp/terraform
cat << EOF > /tmp/terraform/main.tf
variable "variable" {
  description = "Value of the name for the Docker container"
  type        = string
  default     = "the value of the variable"
}

output "variable_output" {
  value = var.variable
}
EOF
cd /tmp/terraform
terraform init &> /dev/null

The following cell invokes the configuration output.

In [25]:
terraform output

variable_output = "the value of the variable"


Pass the value you like your configuration to run with to the `terraform apply` command with `-var` parameter.

In [27]:
terraform apply -auto-approve -var "variable=custom value"


Changes to Outputs:
  ~ variable_output = "the value of the variable" -> "custom value"

You can apply this plan to save these new output values to the Terraform state,
without changing any real infrastructure.

Apply complete! Resources: 0 added, 0 changed, 0 destroyed.

Outputs:

variable_output = "custom value"
